## balanced_profolio

In [3]:
import pandas as pd
import numpy as np

In [33]:
target.head()

,Date,sc_1d_fwd_ret,mom_1d_fwd_ret,value_1d_fwd_ret,sp_1d_fwd_ret,sc_1d_fwd_rel_ret,mom_1d_fwd_rel_ret,value_1d_fwd_rel_ret,sc_1d_fwd_d,mom_1d_fwd_d,value_1d_fwd_d,sc_1d_fwd_rel_d,mom_1d_fwd_rel_d,value_1d_fwd_rel_d
0,2000-05-30,0.003626,-0.002746,-0.005446,-0.001301,0.004926,-0.001446,-0.004145,1,0,0,1,0,0
1,2000-05-31,0.022660,0.029970,0.024081,0.019858,0.002802,0.010112,0.004223,1,1,1,1,1,1
2,2000-06-01,0.052023,0.033394,0.016557,0.019637,0.032386,0.013757,-0.003080,1,1,1,1,1,0
3,2000-06-02,-0.003663,-0.001076,-0.001554,-0.006519,0.002856,0.005443,0.004965,0,0,0,1,1,1
4,2000-06-05,0.009804,-0.014203,-0.012033,-0.006671,0.016475,-0.007532,-0.005362,1,0,0,1,0,0


In [4]:
path = '/Users/yutingmei/Projects/APlusBernstein-Project/data/raw/FactorTimingData-ABVU2022.xlsx'
target = pd.read_excel(path, sheet_name = 'target')

In [ ]:
def balanced_profolio(target, pred):
    target_ = target.iloc[:, 1:4]
    start = 100000
    initial = 100000
    s_p = target.iloc[:, 4]
    real = target.iloc[:, -3:]
    compare_result = []
    for i in range(pred.shape[0]):
        row = target_.iloc[i, :]
        start = sum(list(map(lambda x: (start / 3) * (x - 1), row)))
        if pred.iloc[i,:].sum() >= 2:
            op_invest = 1
            op_s_p = -1
        else:
            op_invest = -1
            op_s_p = 1
        if real.iloc[i, :].sum() >= 2:
            op_real = 1
        else:
            op_real = -1
        if op_real != op_invest:
            op_final = -1
        else:
            op_final = 1
        result_pred = (op_invest * start + op_s_p * s_p[i] * initial) * op_final
        compare_result.append(result_pred)
    return compare_result